# Donkey Car Attempt

My attempts to build a CNN from scratch are hitting walls. I am going to try out a model suggested on the [Donkey Car Webpage](https://docs.donkeycar.com/dev_guide/model/), to see if it's any better.

In [1]:
## Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
import time
import pickle

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras_tuner import BayesianOptimization, Hyperband, HyperModel

from os.path import exists

from tensorflow import keras
from tensorflow.keras.backend import concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, Flatten, Convolution2D
from tensorflow.keras.metrics import MAE, MSE, RootMeanSquaredError
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2021-11-17 14:38:14.204946: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
## Constants
working_date = '11_12_2021'
working_time = '19_28_18'

model_history_file = '../models/model_history.csv'


dual_outputs = False
BATCH_SIZE = 128
batch_sizes = [32, 64, 128, 256, 512, 1024, 2048]
early_stop_patience = None # None for no stop
epochs = 250

In [3]:
## Directories
data_directory = f'../data/{working_date}/{working_time}'
model_directory = f'../models/{working_date}/{working_time}'

## File paths
cam_input_dataset_file = f'{data_directory}/X_img.npy'
telem_input_dataset_file = f'{data_directory}/X_telem.pkl'
target_dataset_file = f'{data_directory}/y.npy'
scaler_file = f'{data_directory}/sc.pkl'

In [4]:
## Make sure model history exists
if not exists(model_history_file):
    model_history = pd.DataFrame(columns=['model', 'history', 'batch_size', 
                                          'r2_score', 'mae_score',
                                          'mse_score', 'rmse_score'])
else:
    model_history = pd.read_csv(model_history_file, index_col=0)

## Data

### Load Datasets

In [5]:
## Load the datasets
X_cam = np.load(cam_input_dataset_file)
X_telem = pd.read_pickle(telem_input_dataset_file).to_numpy()
y = np.load(target_dataset_file)

In [6]:
## Split for dual output
if dual_outputs:
    y_steering = y[:, 0]
    y_throttle = y[:, 1]

In [7]:
X_cam.shape

(102368, 64, 64, 1)

### Train-Test Split

In [8]:

if dual_outputs:
    datasets = train_test_split(X_cam, X_telem, y_steering, y_throttle, test_size=0.1, random_state=0)
else:
    datasets = train_test_split(X_cam, X_telem, y, test_size=0.1, random_state=0)
    
X_cam_train = datasets[0]
X_cam_test = datasets[1]
X_telem_train = datasets[2]
X_telem_test = datasets[3]
y_train = datasets[4]
y_test = datasets[5]
    
if dual_outputs:
    y_st_train = datasets[4]
    y_st_test = datasets[5]
    y_th_train = datasets[6]
    y_th_test = datasets[7]
# else:
#     y_train = datasets[4]
#     y_test = datasets[5]

### Scale IMU Data

In [9]:
# ss = StandardScaler()
ss = MinMaxScaler() # default range (0, 1)
X_telem_train_sc = ss.fit_transform(X_telem_train)
X_telem_test_sc = ss.transform(X_telem_test)

### Save the Scaler for Predictions

In [10]:
pickle.dump(ss, open(scaler_file, 'wb'))

In [11]:
cam_input_shape = X_cam_train[0].shape
telem_input_shape = X_telem_train_sc[0].shape

In [12]:
# https://docs.donkeycar.com/dev_guide/model/
# https://github.com/autorope/donkeycar/blob/dev/donkeycar/parts/keras.py

def create_model(cam_input_shape, telem_input_shape):
    drop = 0.2
    img_in = Input(shape=cam_input_shape, name='img_in') 
    x = img_in
    x = conv2d(24, 5, 2, 1)(x)
    x = Dropout(drop)(x)
    x = conv2d(32, 5, 2, 2)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 5, 2, 3)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 3, 1, 4)(x)
    x = Dropout(drop)(x)
    x = conv2d(64, 3, 1, 5)(x)
    x = Dropout(drop)(x)
    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu', name='dense_1')(x)
    x = Dropout(drop)(x)
    x = Dense(50, activation='relu', name='dense_2')(x)
    x = Dropout(drop)(x)
    # up to here, this is the standard linear model, now we add the
    # sensor data to it
    telem_in = Input(telem_input_shape, name='telem_in')
    y = telem_in
    z = concatenate([x, y])
    # here we add two more dense layers
    z = Dense(50, activation='relu', name='dense_3')(z)
    z = Dropout(drop)(z)
    z = Dense(50, activation='relu', name='dense_4')(z)
    z = Dropout(drop)(z)
    
    if dual_outputs:
    # two outputs for angle and throttle
        outputs = [
            Dense(1, activation='linear', name='steering_outputs')(z),
            Dense(1, activation='linear', name='throttle_outputs')(z)
        ]
    else:
        # combined output
        outputs = Dense(2, activation='linear', name='combined_outputs')(z)

    # the model needs to specify the additional input here
    model = Model(inputs=[img_in, telem_in], outputs=outputs)
    return model


def conv2d(filters, kernel, strides, layer_num, activation='relu'):
    """
    Helper function to create a standard valid-padded convolutional layer
    with square kernel and strides and unified naming convention
    :param filters:     channel dimension of the layer
    :param kernel:      creates (kernel, kernel) kernel matrix dimension
    :param strides:     creates (strides, strides) stride
    :param layer_num:   used in labelling the layer
    :param activation:  activation, defaults to relu
    :return:            tf.keras Convolution2D layer
    """
    return Convolution2D(filters=filters,
                         kernel_size=(kernel, kernel),
                         strides=(strides, strides),
                         activation=activation,
                         name='conv2d_' + str(layer_num))

In [13]:
def run_model(model, batch_size, epochs=500, early_stop_patience=5, verbose=0):
    if early_stop_patience:
        stop_early = EarlyStopping(patience=early_stop_patience)
        callbacks = [stop_early]
    else:
        callbacks = None
    ## Fit the best model
    results = model.fit(
        x=[X_cam_train, X_telem_train_sc],
        y=y_train,
        batch_size=BATCH_SIZE, 
        epochs=epochs, 
        callbacks=callbacks,
        validation_data=((X_cam_test, X_telem_test_sc), y_test),
        verbose=verbose
    )
    return model, results

In [14]:
def save_model(model, results, batch_size):
    model_index = max(0, model_history.index.max() + 1)
    model_path = f'{model_directory}/model_{model_index}.h5'
    history_dictionary = {
        'model': model_path,
        'history': results.history,
        'batch_size': batch_size,
    }
    if dual_outputs:
        history_dictionary['mae_score'] = (
            results.history['val_steering_outputs_mae'][-1],
            results.history['val_throttle_outputs_mae'][-1],
        )
        history_dictionary['mse_score'] = (
            results.history['val_steering_outputs_loss'][-1],
            results.history['val_throttle_outputs_loss'][-1]
        )
        history_dictionary['rmse_score'] = (
            results.history['val_steering_outputs_root_mean_squared_error'][-1],
            results.history['val_throttle_outputs_root_mean_squared_error'][-1] 
        )
    else:
        history_dictionary['mae_score'] = (
            results.history['val_mae'][-1],
        )
        history_dictionary['mse_score'] = (
            results.history['val_loss'][-1],
        )
        history_dictionary['rmse_score'] = (
            results.history['val_root_mean_squared_error'][-1],
        )
    model_history = model_history.append(history_dictionary, ignore_index=True)
    ## Saving as h5 for backwards compatibility
    model.save(model_path, save_format='h5')
    model_history.to_csv(model_history_file)

In [15]:
def plot_metrics(results):
    fig, ax1, ax2, ax3 = plt.subplots(1, 3, figsize=(10,15))
    
    ax1.title('Loss (MSE)', size=13)
    ax1.plot(results.history['loss'], label = 'MSE')
    ax1.plot(results.history['val_loss'], label = 'MSE')
    if dual_outputs:
        ax1.plot(results.history['val_steering_outputs_loss'], label = 'Val Str MSE')
        ax1.plot(results.history['val_throttle_outputs_loss'], label = 'Val Thr MSE')
    ax1.legend()
    
    ax2.title('RMSE', size=13)
    if dual_outputs:
        ax2.plot(results.history['val_steering_outputs_root_mean_squared_error'], label = 'Val Str RMSE')
        ax2.plot(results.history['val_throttle_outputs_root_mean_squared_error'], label = 'Val Thr RMSE')
    else:
        ax2.plot(results.history['root_mean_squared_error'], label = 'RMSE')
        ax2.plot(results.history['val_root_mean_squared_error'], label = 'Val RMSE')
    ax2.legend()
    
    ax3.title('MAE', size=13)
    if dual_outputs:
        ax3.plot(results.history['val_steering_outputs_mae'], label = 'Val Str MAE')
        ax3.plot(results.history['val_throttle_outputs_mae'], label = 'Val Thr MAE')
    else:
        ax3.plot(results.history['mae'], label = 'MAE')
        ax3.plot(results.history['val_mae'], label = 'Val MAE')
    ax3.legend()
    
    plt.tight_layous();

## Model Loop

In [16]:
if dual_outputs:
    y_train = (y_st_train, y_th_train)
    y_test = (y_st_test, y_th_test)
else:
    y_train = y_train
    y_test = y_test
    # y_train = np.array((y_st_train, y_th_train)).T
    # y_test = np.array((y_st_test, y_th_test)).T

In [17]:
## Run models for each batch size
print('---')
for batch_size in batch_sizes:
    print(f'Batch size {batch_size} start: {time.strftime("%H:%m")}')
    model = create_model(cam_input_shape, telem_input_shape)
    model.compile(loss='mse', 
                  optimizer=Adam(learning_rate=0.00001), 
                  metrics=['mae', RootMeanSquaredError()])
    model, res = run_model(model, batch_size, 
                           early_stop_patience=early_stop_patience)
    save_model(mod, res, batch_size)
    # plot_metrics(res)
    duration = time.time() - start_time # seconds
    print(f'Batch size {batch_size} end: {time.strftime("%H:%m")}')
    print('---')
model_history.tail(len(batch_sizes))

SyntaxError: f-string: unmatched '(' (2859854858.py, line 4)

## Model, Non-loop

In [ ]:
model = create_model(cam_input_shape, telem_input_shape)
model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=['mae', RootMeanSquaredError()])

### Fit

In [ ]:
stop_early = EarlyStopping(patience=5)

In [ ]:
# # Separate Outputs
if dual_outputs:
    y_train = (y_st_train, y_th_train)
    y_test = (y_st_test, y_th_test)
# else:
    # y_train = np.array((y_st_train, y_th_train)).T
    # y_test = np.array((y_st_test, y_th_test)).T

In [ ]:
## Fit the best model
results = model.fit(
    x=[X_cam_train, X_telem_train_sc],
    y=y_train,
    batch_size=BATCH_SIZE, 
    epochs=500, 
    # callbacks=[stop_early],
    validation_data=((X_cam_test, X_telem_test_sc), y_test),
    verbose=0
)

In [ ]:
plt.plot(results.history['loss'], label = 'Loss')
plt.plot(results.history['val_loss'], label = 'Val Loss')
if dual_outputs:
    plt.plot(results.history['val_steering_outputs_loss'], label = 'Val Str Loss')
    plt.plot(results.history['val_throttle_outputs_loss'], label = 'Val Thr Loss')
plt.legend();

In [ ]:
if dual_outputs:
    plt.plot(results.history['val_steering_outputs_mae'], label = 'Val Str MAE')
    plt.plot(results.history['val_throttle_outputs_mae'], label = 'Val Thr MAE')
else:
    plt.plot(results.history['mae'], label = 'MAE')
    plt.plot(results.history['val_mae'], label = 'Val MAE')
plt.legend();

In [ ]:
if dual_outputs:
    plt.plot(results.history['val_steering_outputs_root_mean_squared_error'], label = 'Val Str RMSE')
    plt.plot(results.history['val_throttle_outputs_root_mean_squared_error'], label = 'Val Thr RMSE')
else:
    plt.plot(results.history['root_mean_squared_error'], label = 'RMSE')
    plt.plot(results.history['val_root_mean_squared_error'], label = 'Val RMSE')
plt.legend();

## Save model

In [ ]:
model_index = max(0, model_history.index.max() + 1)
model_path = f'{model_directory}/model_{model_index}.h5'

history_dictionary = {
    'model': model_path,
    'history': results.history,
}

if dual_outputs:
    history_dictionary['mae_score'] = (
        results.history['val_steering_outputs_mae'][-1],
        results.history['val_throttle_outputs_mae'][-1],
    )
    history_dictionary['mse_score'] = (
        results.history['val_steering_outputs_loss'][-1],
        results.history['val_throttle_outputs_loss'][-1]
    )
    history_dictionary['rmse_score'] = (
        results.history['val_steering_outputs_root_mean_squared_error'][-1],
        results.history['val_throttle_outputs_root_mean_squared_error'][-1] 
    )
else:
    history_dictionary['mae_score'] = (
        results.history['val_mae'][-1],
    )
    history_dictionary['mse_score'] = (
        results.history['val_loss'][-1],
    )
    history_dictionary['rmse_score'] = (
        results.history['val_root_mean_squared_error'][-1],
    )

model_history = model_history.append(history_dictionary, ignore_index=True)
model_history.tail()

In [ ]:
## Saving as h5 for backwards compatibility
model.save(model_path, save_format='h5')
model_history.to_csv(model_history_file)

In [ ]:
model_path